# Optimising compute with concurrent IO in OpenCL

With many iterative processes there is a need to get information **off** the device at regular intervals. Up to this point we have been transferring data off the compute device **after** kernel execution. Also, the routines to read information from device buffers have thus far been used in a blocking manner, that is the program pauses while the read occurs. Most compute devices have the ability to transfer data **while** kernels are being executed. This means IO transfers can take place during compute and may in some instances **take place entirely** during kernel execution. For the cost of additional programming complexity, significant compute savings can be obtained, as the following diagram illustrates:

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/optimising_io.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: The difference between sequential and concurrent IO.</figcaption>
</figure>

## Concurrent IO is enabled with multiple command queues

The **key to leveraging concurrent IO** is to have one command queue for the kernels and one or more command queues for moving data. Then IO operations can take place largely independently of the compute operations. OpenCL Events and the clFinish command can help establish dependencies between command queues. Non-blocking IO calls may also help with concurrency.

## Example with the 2D wave equation

The [scalar wave equation](https://en.wikipedia.org/wiki/Wave_equation) adequately describes a number of different wave phenomena. If **U** is a 2D grid storing the amplitude of the wave at every location (the wavefield), **V** is a 2D grid storing velocity, and **t** is time, then 2D waves propagate according to the formula,

$$\frac{\partial^2 \textbf{U}}{{\partial t}^2}=\textbf{V}^2 \left (\frac{\partial^2 \textbf{U}}{{\partial x_{0}}^2}+\frac{\partial^2 \textbf{U}}{{\partial x_{1}}^2} \right)+f(t)$$

where $x_0$ and $x_1$ are spatial directions and $f(t)$ is a forcing term. If $\Delta t$ is the time step a second-order finite-difference approximation to the time derivative is given in terms of the amplitude at timesteps $\textbf{U}_{0}, \textbf{U}_{1}$ and $\textbf{U}_{2}.$ 

$$\frac{\partial^2 \textbf{U}}{{\partial t}^2} \approx \frac{1}{\Delta t^2} \left ( \textbf{U}_{0} -2 \textbf{U}_{1}+\textbf{U}_{2} \right ) $$

Replace $\frac{\partial^2 \textbf{U}}{{\partial t}^2}$ with $\frac{1}{\Delta t^2} \left( \textbf{U}_{0} -2 \textbf{U}_{1}+\textbf{U}_{2} \right )$ and solve for $\textbf{U}_{2}$.

$$\textbf{U}_{2} \approx 2 \textbf{U}_{1} - \textbf{U}_{0} + \Delta t^2\textbf{V}^2 \left (\frac{\partial^2 \textbf{U}_{1}}{{\partial x_{0}}^2}+\frac{\partial^2 \textbf{U}_{1}}{{\partial x_{1}}^2} \right)+f_{1}$$

This is an iterative formula to generate the amplitude at the next timestep $\textbf{U}_2$ if we know the present ampltiude $\textbf{U}_{1}$ and past amplitude $\textbf{U}_{0}.$ We also use finite difference approximations for the spatial derivatives, and express the spatial derivatives as a matrix multiplied by $\textbf{U}_{1}$, but this complexity is unnecessary to show here. All we need to know is that the next timestep is a function ${\textbf{F}}$ of the present and past timesteps, the velocity, and the forcing term.

$$\textbf{U}_{2}=\textbf{F}(\textbf{U}_0, \textbf{U}_1, \textbf{V}, f_{1})$$

> In geophysics we usually use a [Ricker Wavelet](https://wiki.seg.org/wiki/Dictionary:Ricker_wavelet) for the forcing term $f$, and usually inject that wavelet into one cell within the grid as time progresses.

### Kernel implementation

In [kernels.c](kernels.c) is a kernel called **wave2d_4o** that implements the function **F**. OpenCL buffers store $\textbf{U}_{0}, \textbf{U}_{1}, \textbf{U}_{2}$, and $\textbf{V}$ on the compute device.

```C++
// Kernel to solve the wave equation with fourth-order accuracy in space
__kernel void wave2d_4o (
        __global float* U0,
        __global float* U1,
        __global float* U2,
        __global float* V,
        unsigned int N0,
        unsigned int N1,
        float dt2,
        float inv_dx02,
        float inv_dx12,
        // Position, frequency, and time for the
        // wavelet injection
        unsigned int P0,
        unsigned int P1,
        float pi2fm2t2) {    

    // U2, U1, U0, V is of size (N0, N1)
    size_t i0=get_global_id(1); // Slowest dimension
    size_t i1=get_global_id(0); // Fastest dimension
    
    // Required padding and coefficients for spatial finite difference
    const int pad_l=2, pad_r=2, ncoeffs=5;
    float coeffs[ncoeffs] = {-0.083333336f, 1.3333334f, -2.5f, 1.3333334f, -0.083333336f};
    
    // Limit i0 and i1 to the region of U2 within the padding
    i0=min(i0, (size_t)(N0-1-pad_r));
    i1=min(i1, (size_t)(N1-1-pad_r));
    i0=max((size_t)pad_l, i0);
    i1=max((size_t)pad_l, i1);
    
    // Position within the grid as a 1D offset
    long offset=i0*N1+i1;
    
    // Temporary storage
    float temp0=0.0f, temp1=0.0f;
    float tempV=V[offset];
    
    // Calculate the Laplacian
    #pragma unroll
    for (long n=0; n<ncoeffs; n++) {
        // Stride in dim0 is N1        
        temp0+=coeffs[n]*U1[offset+(n*(long)N1)-(pad_l*(long)N1)];
        // Stride in dim1 is 1
        temp1+=coeffs[n]*U1[offset+n-pad_l];
    }
    
    // Calculate the wavefield U2 at the next timestep
    U2[offset]=(2.0f*U1[offset])-U0[offset]+((dt2*tempV*tempV)*(temp0*inv_dx02+temp1*inv_dx12));
    
    // Inject the forcing term at coordinates (P0, P1)
    if ((i0==P0) && (i1==P1)) {
        U2[offset]+=(1.0f-2.0f*pi2fm2t2)*exp(-pi2fm2t2);
    }
    
}
```

### Problem setup

For this problem we create the 2D grid as a square box of size $(N0,N1)=(256,256)$. The velocity is uniform at 343m/s. This is approximately the speed of sound in air. Then we use a Ricker wavelet as a forcing term to 'let off a firework' in the middle of the box and run a number of timesteps to see how a sound wave propagates in the box. 

<figure style="margin-bottom 3em; margin-top: 2em; margin-left:auto; margin-right:auto; width:100%">
    <img style="vertical-align:middle" src="../images/wave2d_problem.svg"> <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">Figure: Problem setup for the 2D wave equation.</figcaption>
</figure>


At each timestep the kernel **wave2d_4o** is used to update the solution to the next timestep. Old wavefields that are no longer needed are recycled for efficiency.

The code below just writes the velocity array to disk, ready to be read in by the wave codes below.

In [37]:
%matplotlib widget

import os
import sys
import numpy as np
import subprocess
from ipywidgets import widgets
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

sys.path.insert(0, os.path.abspath("../include"))

import py_helper

float_type = np.float32

defines=py_helper.load_defines("mat_size.hpp")

# Velocity of the medium
vel=343.0

# Make up the velocity and first two timesteps
V=np.ones((defines["N0"], defines["N1"]), dtype=float_type)*vel

# write files to disk
V.tofile("array_V.dat")

### Sequential IO solution

In [wave2d_sync.cpp](wave2d_sync.cpp) we use an array of three OpenCL buffers to represent the wavefield at timesteps (0,1,2). A single command queue is used for both kernel execution and IO.

## Run the wave application with synchronous IO

In [39]:
# Run the application
subprocess.run([os.path.join(os.getcwd(),"wave2d_sync.exe"), "-gpu"])

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001166, Vmax=343.000000
dt=0.00116618, fm=34.3, Vmax=343, dt2=1.35998e-06
The synchronous calculation took 43571 milliseconds.


CompletedProcess(args=['/home/toby/Pelagos/Projects/OpenCL_Course/course_material/L9_IO_Optimisation/wave2d_sync.exe', '-gpu'], returncode=0)

## Run the wave application with concurrent IO

In [29]:
# Run the application
subprocess.run([os.path.join(os.getcwd(),"wave2d_async.exe"), "-gpu"])

	               name: gfx1035 
	 global memory size: 536 MB
	    max buffer size: 456 MB
	     max local size: (1024,1024,1024)
	     max work-items: 256
dt=0.001201, Vmax=333.000000
dt=0.0012012, fm=33.3, Vmax=333, dt2=1.44288e-06
The asynchronous calculation took 28995 milliseconds.

CompletedProcess(args=['/home/toby/Pelagos/Projects/OpenCL_Course/course_material/L9_IO_Optimisation/wave2d_async.exe', '-gpu'], returncode=0)

In [15]:
# Read the outputfile back in for display
output=np.fromfile("array_out.dat", dtype=float_type)
nslices=output.size//(defines["N0"]*defines["N1"])
output=output.reshape(nslices, defines["N0"], defines["N1"])

In [16]:
# Animate the result
fig, ax = plt.subplots(1,1, figsize=(8,6))
extent=[ -0.5*defines["D1"], (defines["N1"]-0.5)*defines["D1"],
    -0.5*defines["D0"], (defines["N0"]-0.5)*defines["D0"]]
img = ax.imshow(
    output[0,...], 
    extent=extent, 
    vmin=np.min(output), 
    vmax=np.max(output),
    origin="lower"
)

ax.set_xlabel("Dimension 1")
ax.set_ylabel("Dimension 0")
ax.set_title("Wavefield")

def update(n=0):
    img.set_data(output[n,...])
    plt.show()
    #return (img,)

# Run the interaction
result = widgets.interact(
    update,
    n=(0, output.shape[0]-1, 1)
)

interactive(children=(IntSlider(value=0, description='n', max=639), Output()), _dom_classes=('widget-interact'…